# SSINS Tutorial
* tutorial link: https://ssins.readthedocs.io/en/latest/tutorial.html?highlight=tutorial


* using the order parameter: Order is not a possible key word for ins.mean_subtract. The help function shows it only takes a slice which is usually not set by the user. Tried: removing the argument (worked) now there is an issue with the INS function call. Resolved: just needed to change the inpath to the correct file


* using polt_lib: removed an argument from ins.mean_subtract


* Skipped: writing an MWAF file


### Initializing an SS Object

In [1]:
from SSINS import SS
from SSINS import INS
import numpy as np
from SSINS import Catalog_Plot as cp
import os
from matplotlib import cm
from SSINS import plot_lib
import matplotlib.pyplot as plt
%matplotlib inline
from SSINS import MF

ModuleNotFoundError: No module named 'SSINS'

In [21]:
# The SS object is a subclass of a UVData object and is initialized in an identical way
#omits auto-correlation on read
ss = SS()

In [22]:
#dictionary of atributes
ss.__dict__

{'_Ntimes': <pyuvdata.parameter.UVParameter at 0x7ff783a74320>,
 '_Nbls': <pyuvdata.parameter.UVParameter at 0x7ff78283fc88>,
 '_Nblts': <pyuvdata.parameter.UVParameter at 0x7ff78283fe80>,
 '_Nfreqs': <pyuvdata.parameter.UVParameter at 0x7ff78283fb70>,
 '_Npols': <pyuvdata.parameter.UVParameter at 0x7ff783a9e7b8>,
 '_data_array': <pyuvdata.parameter.UVParameter at 0x7ff783acdb70>,
 '_vis_units': <pyuvdata.parameter.UVParameter at 0x7ff7812c85c0>,
 '_nsample_array': <pyuvdata.parameter.UVParameter at 0x7ff7812c8470>,
 '_flag_array': <pyuvdata.parameter.UVParameter at 0x7ff7812c8cf8>,
 '_Nspws': <pyuvdata.parameter.UVParameter at 0x7ff7812c8ac8>,
 '_spw_array': <pyuvdata.parameter.UVParameter at 0x7ff7812c8668>,
 '_uvw_array': <pyuvdata.parameter.UVParameter at 0x7ff7812c8c88>,
 '_time_array': <pyuvdata.parameter.UVParameter at 0x7ff7812b98d0>,
 '_lst_array': <pyuvdata.parameter.UVParameter at 0x7ff7812b9c18>,
 '_ant_1_array': <pyuvdata.parameter.UVParameter at 0x7ff780826390>,
 '_ant_2_

In [23]:
# This is an MWA uvfits file trimmed to a single pol, 99 baselines, and only
# half of the observing time. We can pass the same keywords for UVData.read to SS.read.
# For instance, we read in only cross correlations here.
ss.read('/home/kelcey/hera/Washington/SSINS/SSINS/data/1061313128_99bl_1pol_half_time.uvfits')

In [24]:
# We time-difference the visibilities during initialization.
# We adjust the attributes during differencing to ensure the SS object is still a UVData object
ss.check()

True

### Applying Flags

In [41]:
inpath = '/home/kelcey/hera/Washington/SSINS/SSINS/data/1061313128_99bl_1pol_half_time.uvfits'
#SSINS uses  masked numpy arrays, so we can work with multiple sets of flags along with original flagged array
#Here we use the original flag, proogating through distancing
ss.read(inpath, flag_choice='original', ant_str='cross')
np.all(ss.data_array.mask == ss.flag_array)

True

## INS Initalization

Making an ins (incoherent noise spectrum)

In [26]:
#To create ins: either pass it the ss previously created or give it the file path we desire. The propper method
#depends on which file we want to use.
ins = INS(ss)

prefix = '/home/kelcey/hera/Washington/SSINS/SSINS/data/tutorial_'
#write the data to this file(clobber will over-write previous data)
ins.write(prefix, output_type='data', clobber = True)
#write the z scores
ins.write(prefix, output_type='z_score', clobber = True)
#write masks to the data
ins.write(prefix, output_type='mask', clobber = True)

File /home/kelcey/hera/Washington/SSINS/SSINS/data/tutorial__SSINS_data.h5 exists; clobbering
File /home/kelcey/hera/Washington/SSINS/SSINS/data/tutorial__SSINS_z_score.h5 exists; clobbering
File /home/kelcey/hera/Washington/SSINS/SSINS/data/tutorial__SSINS_mask.h5 exists; clobbering


Writing time propogated flags

In [27]:
#calculate the time-propogating flags with the mask_to_flag and extend them back across the time difference
tp_flags = ins.mask_to_flags()
#tp_flags is a flag array with length of original data. 
#Any samples that would have contributed to a flag difference are now flagged.
ins.write(prefix, output_type='flags', clobber = True)
#This writes these flags out to a format that can be read by UV flag. This method is automatically called when 
#writing flags, which is  different than writing the masked data.

File /home/kelcey/hera/Washington/SSINS/SSINS/data/tutorial__SSINS_flags.h5 exists; clobbering


In [39]:
#zeroes_like returns array of zeroes with same shape and type as given array
#ss.apply_flags:appliese a custom flag leaving the original flag array untouched so we can revisit it. We
#are only changing the mask on the data.
custom = np.zeros_like(ss.flag_array)
custom[:, 0, 0, :] = 1
ss.apply_flags(flag_choice='custom', custom=custom)
#Now we apply a mask from a generated ins, created above.

### Sky Subtract Plotting by Invoking Catalog Plot

In [40]:
#This cell just un-flags the data.
ss.apply_flags(flag_choice='custom', INS=ins)
np.any(ss.data_array.mask)

Custom flags were chosen, but custom flags were None type. Setting flags to None.


False

In [30]:
#Hstogram of sky-subtracted visibility amplitudes
#VDH_plot: visibility difference histogram
cp.VDH_plot(ss, prefix, file_ext='png', xlabel='Amplitude (~Jy)', xscale='log',
            yscale='log', pre_flag=True, post_model=True)

No handles with labels found to put in legend.


### Incoherent Noise Spectrum with the Mean Subtract method

In [31]:
ins.data_ms = ins.mean_subtract()

# A slice of the array can be calculated by using the f keyword (f for frequencies)
# Set up a slice object for frequency channels 100 to 199 inclusive

#NEW: f is now freq_slice in documentation
f = slice(100, 200)
ins.data_ms[:, f, :] = ins.mean_subtract(f=f)

using the order parameter

In [32]:
# Sometimes the mean appears to drift in time to linear or higher order
# A polynomial fit to each channel can be constructed using the order parameter
ins.order = 2
#make a quadratic fit for each channel
ins.data_ms = ins.mean_subtract()
inpath1 = '/home/kelcey/hera/Washington/SSINS/SSINS/data/1061313128_99bl_1pol_half_time_SSINS.h5'
#make a linear fit
ins = INS(inpath1, order=1)

### Plotting the Incoherent Noise Spectrum

with catalog_plot

In [33]:
#making tickables with the frequency array
prefix = '/home/kelcey/hera/Washington/SSINS/SSINS/data/tutorial_'
xticks = np.arange(0, len(ins.freq_array), 50)
xticklabels = ['%.1f' % (ins.freq_array[tick] * 10 ** (-6)) for tick in xticks]
cp.INS_plot(ins, prefix, data_cmap=cm.plasma, vmin=0, vmax=150, ms_vmin=-5,ms_vmax=5, 
            xticks=xticks, xticklabels=xticklabels,xlabel='Frequency (Mhz)')


using plot_lib

In [42]:
#this  aproach give finer control over which plots are output.
from matplotlib import use
fig, ax = plt.subplots(nrows=2)
prefix = '/home/kelcey/hera/Washington/SSINS/SSINS/data/'

# Here we take an INS and plot its mean-subtracted data in the first
# polarization with different order parameters

for i in range(2):
    ins.order = i
    ins.metric_ms = ins.mean_subtract()
    plot_lib.image_plot(fig, ax[i], ins.metric_ms[:,  :, 0], cmap=cm.coolwarm, 
                      title='order = %i' % i, vmin=-10, vmax=102)
fig.savefig('%s/tutorial_order_compare.png' % prefix)
plt.show()

Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


# Matching Filters

initialization

In [35]:
#Initialization sets desired parameters, RFI shapes are passed with a dictionary, 
shape_dict = {'TV6': [1.74e8, 1.81e8],
                  'TV7': [1.81e8, 1.88e8],
                  'TV8': [1.88e8, 1.96e8]}
#determines the max strength of  the outliers to leave unflagged
sig_thresh = 5

#Single frequency flagger
narrow = False

#broadband streak flagger
streak = False

#initalizing with the frequency array
mf = MF(ins.freq_array, sig_thresh, shape_dict=shape_dict, narrow=narrow, streak=streak)

### Applying matching fiter tests

basic match shape test

In [36]:
#apply flags to samples matching the flagging criteria
mf.apply_match_test(ins, event_record=True)

#plot the result and apply the mask
cp.INS_plot(ins, prefix, ms_vmin=-mf.sig_thresh, ms_vmax=mf.sig_thresh)
#writing the match events out to a file
ins.write(prefix, output_type='match_events')

Beginning match_test at 15:06:01
Finished match_test at 15:06:01


flagging times for highy contaminated channels

In [37]:
#n_samp_thresh set on initalization, if there are less than these n_thresh channells, all chcannels are flagged.
mf = MF(ins.freq_array, sig_thresh=5, N_samp_thresh=20)
mf.apply_match_test(ins, apply_samp_thresh=True)

Beginning match_test at 15:06:02
Finished match_test at 15:06:02


find the fraction  of logged data

In [38]:
#this is the total fraction of flagged data. The mask on the INS data keeps track of what is flagged.
print(np.mean(ins.metric_array.mask))

0.2674575617283951
